# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from pathlib import Path
import warnings
import pickle

from src.utils.coco import COCODataset
from src.utils.pretty_print import *
from src.models import get_model_resnet34, get_model_unet, get_model_maskrcnn_with_groupnorm
from src.utils.eval import resnet_eval, unet_eval, maskrcnn_eval

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

SEED = 42

# for reproducibility
torch.manual_seed(SEED)

# Datasets

## Slide dataset

In [3]:
slide_path = Path("/storage01/bolma/dev/data/datasets/WSI-ROI/slides/l8/annotations.json")

In [4]:
slide_dataset = COCODataset(
    annotation_file=slide_path,
    train=False,
    transform=None,
    random_seed=SEED,
    split_file=Path("dataset_split.json"),
    bbox_format='pascal_voc'
)

print_success("Datasets is ready to use!")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

✅ Loaded split from dataset_split.json

✅ Found 38 training images and 10 test images

✅ Loaded test set with 10 images


✅ Datasets is ready to use!


In [5]:
BATCH_SIZE = 1
NUM_WORKERS = 8

In [6]:
slide_dataloader = DataLoader(
    slide_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    #collate_fn=lambda x: tuple(zip(*x))
)

print_info(f"Created DataLoader with batch size {BATCH_SIZE}")
print(f"\t- Test batches: {len(slide_dataloader)}")


ℹ️ Created DataLoader with batch size 1
	- Test batches: 10


## Tile dataset

In [33]:
tile_path = Path("/storage01/bolma/dev/data/datasets/WSI-ROI/tiles/l6_256x256_tiles.json")

In [34]:
tile_dataset = COCODataset(
    annotation_file=tile_path,
    train=False,
    transform=None,
    random_seed=SEED,
    split_file=Path("dataset_split.json"),
    bbox_format='pascal_voc'
)

print_success("Datasets is ready to use!")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

✅ Loaded split from dataset_split.json

✅ Found 4423 training images and 1211 test images

✅ Loaded test set with 1211 images


✅ Datasets is ready to use!


In [35]:
BATCH_SIZE = 32
NUM_WORKERS = 8

In [36]:
def classification_collate_fn(batch):
    """
    Custom collate function for to properly handle batching for classification tasks.
    """
    images = []
    labels = []
    
    for sample in batch:
        image, target = sample
        images.append(image)
        
        # For classification, we only need the label
        # Assuming the first label is the class we want to predict
        if 'labels' in target and len(target['labels']) > 0:
            labels.append(target['labels'][0])
        else:
            # Default to class 0 if no label is present
            labels.append(torch.tensor(0))
    
    # Stack images and labels into batches
    images = torch.stack(images, 0)
    labels = torch.stack(labels, 0)
    
    return images, {'labels': labels}

In [37]:
tile_dataloader = DataLoader(
    tile_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=classification_collate_fn
)

print_info(f"Created DataLoader with batch size {BATCH_SIZE}")
print(f"\t- Test batches: {len(tile_dataloader)}")


ℹ️ Created DataLoader with batch size 32
	- Test batches: 38


# Evaluation

## Tile-based

In [38]:
resnet = get_model_resnet34(num_classes=2, pretrained=True)

optimizer = torch.optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9)

Using cache found in /home/bolma/.cache/torch/hub/pytorch_vision_v0.10.0


In [39]:
resnet.load_state_dict(torch.load("src/models/weights/ResNet_trained_weights_eval.pth", weights_only=True))
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
resnet_results = resnet_eval(model=resnet,device=device, eval_dataset_annotation_path=tile_path)

results_path = Path("results/resnet_results.pkl")
    
with open(results_path, 'wb') as f:
    pickle.dump(resnet_results, f)
print_success(f"Results saved to {results_path}")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

✅ Loaded split from /storage01/bolma/dev/tumor-region-detection-thesis/dataset_split.json

✅ Found 4423 training images and 1211 test images

✅ Loaded test set with 1211 images



Processing images: 100%|██████████| 1211/1211 [00:05<00:00, 226.29it/s]


Found 10 unique WSI sources.

✅ Results saved to results/resnet_results.pkl


In [41]:
print(f"Mean IoU = {np.average(np.array(resnet_results['ious'])):.4f}, Mean inference time = {np.average(np.array(resnet_results['inference_times'])):.4f} s")

Mean IoU = 0.4954, Mean inference time = 2.7725 s


## Semantic segmentation

In [10]:
unet = get_model_unet(in_channels=3,
                          out_channels=1,
                          features=[32, 64, 128, 256])

In [11]:
unet.load_state_dict(torch.load("src/models/weights/UNet_trained_weights_eval.pth", weights_only=True))
unet.eval()

UNet(
  (ups): ModuleList(
    (0): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (2): ConvTranspose2d(256, 128, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), pad

In [32]:
# Supressing some warning coming from a library with legacy code
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', message='Creating a tensor from a list of numpy.ndarrays is extremely slow.')
    unet_results = unet_eval(
        model=unet,
        device=device,
        dataloader=slide_dataloader,
)
    
results_path = Path("results/unet_results.pkl")
    
with open(results_path, 'wb') as f:
    pickle.dump(unet_results, f)
print_success(f"Results saved to {results_path}")

10it [00:00, 10.11it/s]


✅ Results saved to results/unet_results.pkl


In [27]:
print(f"Mean IoU = {np.average(np.array(unet_results['ious'])):.4f}, Mean inference time = {np.average(np.array(unet_results['inference_times'])):.4f} s")

Mean IoU = 0.4837, Mean inference time = 0.0027 s


## Instance segmentation

In [42]:
maskrcnn = get_model_maskrcnn_with_groupnorm(num_classes=2, 
                      box_score_thresh=0.1,
                      box_nms_thresh=0.3,
                      box_fg_iou_thresh=0.5,
                      box_bg_iou_thresh=0.4,
                      box_detections_per_img=10,
                      image_mean = [0.485, 0.456, 0.406],
                      image_std = [0.229, 0.224, 0.225])

In [43]:
maskrcnn.load_state_dict(torch.load("src/models/weights/MaskRCNN_trained_weights_eval.pth", weights_only=True))
maskrcnn.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): GroupNorm(32, 64, eps=1e-05, affine=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): GroupNorm(32, 64, eps=1e-05, affine=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): GroupNorm(32, 256, eps=1

In [44]:
# Supressing some warning coming from a library with legacy code
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', message='Creating a tensor from a list of numpy.ndarrays is extremely slow.')
    maskrcnn_results = maskrcnn_eval(
        model=maskrcnn,
        device=device,
        dataloader=slide_dataloader,
)
    
results_path = Path("results/maskrcnn_results.pkl")
    
with open(results_path, 'wb') as f:
    pickle.dump(maskrcnn_results, f)
print_success(f"Results saved to {results_path}")

10it [00:01,  8.83it/s]



✅ Results saved to results/maskrcnn_results.pkl


In [45]:
print(f"Mean IoU = {np.average(np.array(maskrcnn_results['ious'])):.4f}, Mean inference time = {np.average(np.array(maskrcnn_results['inference_times'])):.4f} s")

Mean IoU = 0.4539, Mean inference time = 0.0871 s


# Afterwork

## Matching batch indices with WSI names for model comparison

In [73]:
def map_indices_to_wsi_names(slide_dataset):
    """Maps dataloader batch indices to WSI filenames/identifiers"""
    index_to_wsi_map = {}
    
    # Iterate through dataset using its indices
    for idx in range(len(slide_dataset)):
        # Get the image info - accessing the internal COCO dataset
        img_id = slide_dataset.image_ids[idx]
        img_info = slide_dataset.coco.loadImgs(img_id)[0]
        
        # Extract filename from the image path
        file_path = img_info['file_name']
        wsi_name = Path(file_path).stem
        
        # Create mapping from index to WSI name
        index_to_wsi_map[idx] = wsi_name
    
    return index_to_wsi_map

index_to_wsi_map = map_indices_to_wsi_names(slide_dataset)

print(index_to_wsi_map)

{0: '8394_24_A4_lvl8', 1: '1170_17_HE_40x_lvl8', 2: '779-21_a2_HE_20230308_8_lvl8', 3: '6766_21_A3_HE_20240515_lvl8', 4: '290_23_A3_HE_20240116_1_lvl8', 5: '109_17_HE_40x_lvl8', 6: '92_17_HE_40x_lvl8', 7: '5003_11_HE_40x_lvl8', 8: '995_18_HE_40x_lvl8', 9: '3201_17_HE_40x_lvl8'}


In [78]:
matched_results = {
    'method': [],
    'wsi_name': [],
    'iou': [],
    'inference_time': [],
    'prediction': [],
}

# ResNet results
for i, wsi_name in enumerate(resnet_results['indices']):
    matched_results['method'].append('Tile-based')
    matched_results['wsi_name'].append(wsi_name)
    matched_results['iou'].append(resnet_results['ious'][i])
    matched_results['inference_time'].append(resnet_results['inference_times'][i])
    matched_results['prediction'].append(resnet_results['predictions'][i])
# UNet results
for i, idx in enumerate(unet_results['indices']):
    if idx in index_to_wsi_map:
        matched_results['method'].append('Semantic segmentation')
        matched_results['wsi_name'].append(index_to_wsi_map[idx])
        matched_results['iou'].append(unet_results['ious'][i])
        matched_results['inference_time'].append(unet_results['inference_times'][i])
    else:
        raise ValueError(f"Index {idx} not found in index_to_wsi_map.")
# Mask R-CNN
for i, idx in enumerate(maskrcnn_results['indices']):
    if idx in index_to_wsi_map:
        matched_results['method'].append('Instance segmentation')
        matched_results['wsi_name'].append(index_to_wsi_map[idx])
        matched_results['iou'].append(maskrcnn_results['ious'][i])
        matched_results['inference_time'].append(maskrcnn_results['inference_times'][i])
    else:
        raise ValueError(f"Index {idx} not found in index_to_wsi_map.")
    
# Save the matched results to a pickle file
results_path = Path("results/matched_results.pkl")
with open(results_path, 'wb') as f:
    pickle.dump(matched_results, f)
print_success(f"Results saved to {results_path}")


✅ Results saved to results/matched_results.pkl


# Results

## Loading pickled data (no need to rerun the whole eval process)

In [79]:
with open('results/matched_results.pkl', 'rb') as f:
    results = pickle.load(f)

print_success("All results loaded successfully!")


✅ All results loaded successfully!


## Results printing

In [60]:
maskrcnn_mean_iou = np.average(np.array(maskrcnn_results['ious']))
maskrcnn_mean_inference_time = np.average(np.array(maskrcnn_results['inference_times']))
unet_mean_iou = np.average(np.array(unet_results['ious']))
unet_mean_inference_time = np.average(np.array(unet_results['inference_times']))
resnet_mean_iou = np.average(np.array(resnet_results['ious']))
resnet_mean_inference_time = np.average(np.array(resnet_results['inference_times']))

In [61]:
# Average statistics
print("\nAverage statistics:")
print("-" * 60)
print(f"{'Method':<25} {'Inference Time (s/image)':<28} {'IoU':<20}")
print("-" * 60)
print(f"{'Tile-based':<25} {resnet_mean_inference_time:<28.4f} {resnet_mean_iou:<10.2f}")
print(f"{'Semantic segmentation':<25} {unet_mean_inference_time:<28.4f} {unet_mean_iou:<10.2f}")
print(f"{'Instance segmentation':<25} {maskrcnn_mean_inference_time:<28.4f} {maskrcnn_mean_iou:<10.2f}")

# Individual iou values
resnet_ious = [float(iou) for iou in resnet_results['ious']]
unet_ious = [float(iou) for iou in unet_results['ious']]
maskrcnn_ious = [float(iou) for iou in maskrcnn_results['ious']]

print("\nIndividual IoU values:")
print("-" * 60) 
print(f"{'Method':<25} {'IoU':<20}")
print("-" * 60) 
print(f"{'Tile-based':<25} {[f'{iou:.2f}' for iou in resnet_ious]}")
print(f"{'Semantic segmentation':<25} {[f'{iou:.2f}' for iou in unet_ious]}")
print(f"{'Instance segmentation':<25} {[f'{iou:.2f}' for iou in maskrcnn_ious]}")


Average statistics:
------------------------------------------------------------
Method                    Inference Time (s/image)     IoU                 
------------------------------------------------------------
Tile-based                2.7725                       0.50      
Semantic segmentation     0.0027                       0.48      
Instance segmentation     0.0871                       0.45      

Individual IoU values:
------------------------------------------------------------
Method                    IoU                 
------------------------------------------------------------
Tile-based                ['0.62', '0.43', '0.57', '0.74', '0.69', '0.73', '0.00', '0.20', '0.43', '0.54']
Semantic segmentation     ['0.00', '0.66', '0.79', '0.67', '0.26', '0.75', '0.19', '0.32', '0.51', '0.69']
Instance segmentation     ['0.47', '0.70', '0.55', '0.56', '0.31', '0.51', '0.28', '0.30', '0.45', '0.41']


In [ ]:
import pandas as pd
from IPython.display import display

# Create DataFrame for Average Statistics
avg_stats = pd.DataFrame({
    'Method': ['Tile-based', 'Semantic segmentation', 'Instance segmentation'],
    'Inference Time (s/image)': [resnet_mean_inference_time, unet_mean_inference_time, maskrcnn_mean_inference_time],
    'IoU': [resnet_mean_iou, unet_mean_iou, maskrcnn_mean_iou]
})

# Print average statistics with pandas
print("\nAverage statistics:")
display(avg_stats.style.format({
    'Inference Time (s/image)': '{:.4f}',
    'IoU': '{:.2f}'
}))

# Group the matched results by method and calculate statistics
matched_results_statistics = {
    'method': [],
    'wsi_name': [],
    'iou': [],
    'inference_time': [],
}

for key in matched_results_statistics.keys():
    matched_results_statistics[key] = matched_results[key]

results_df = pd.DataFrame(matched_results_statistics)
method_grouped = results_df.groupby('method')

# Calculate summary statistics for each method
summary_stats = method_grouped.agg({
    'iou': ['mean', 'std', 'min', 'max'],
    'inference_time': ['mean', 'std', 'min', 'max']
}).round(4)

print("\nDetailed statistics by method:")
display(summary_stats)

# For individual IoU values, create a more readable DataFrame
iou_by_method = {
    'Tile-based': resnet_ious,
    'Semantic segmentation': unet_ious,
    'Instance segmentation': maskrcnn_ious
}

# Find the maximum length to pad lists
max_len = max(len(ious) for ious in iou_by_method.values())

# Pad lists with NaN values to make them equal length
for method, ious in iou_by_method.items():
    iou_by_method[method] = ious + [float('nan')] * (max_len - len(ious))

# Create DataFrame for individual IoU values
iou_df = pd.DataFrame(iou_by_method)

print("\nIndividual IoU values:")
display(iou_df.style.format('{:.2f}'))

# Optional: Create a boxplot visualization of IoU values by method
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(data=results_df, x='method', y='iou')
plt.title('IoU Distribution by Method')
plt.ylabel('IoU')
plt.xlabel('Method')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Average statistics:


,Method,Inference Time (s/image),IoU
0,Tile-based,2.7725,0.50
1,Semantic segmentation,0.0027,0.48
2,Instance segmentation,0.0871,0.45


ValueError: All arrays must be of the same length